# import

In [ ]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

from tqdm import tqdm
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from glob import glob
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util

%matplotlib inline

# load model

In [ ]:
PATH_TO_MODEL_DIR = 'workspace/exported-models/ssd_resnet_v1'

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# load test image

In [ ]:
submission_df = pd.read_csv('dataset/sample_submission.csv')
submission_df.head()

In [ ]:
def load_image_into_numpy_array(path):
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8), im_width, im_height

In [ ]:
category_index = {
    1: {'id': 1, 'name': 'typical'},
    2: {'id': 2, 'name': 'indeterminate'},
    3: {'id': 3, 'name': 'atypical'}
}

In [ ]:
predicts = []
count = 0

for index, data in tqdm(submission_df.iterrows()):
    if(data['id'].find('study') != -1):
        path = glob('dataset/test/'+data['id'][:-6]+'/*/*.jpg')[0]

    else:
        path = glob('dataset/test/*/*/'+data['id'][:-6]+'.jpg')[0]

    (image_np, width, height) = load_image_into_numpy_array(path)
    input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    if(data['id'].find('study') != -1):
        str_pred = []
        for index in range(len(detections['detection_scores'][0])):
            if(detections['detection_scores'][0].numpy()[index] > 0.5):
                str_pred.append(category_index[detections['detection_classes'][0].numpy(
                ).astype(np.int32)[index]]['name'])
                str_pred.append(
                    str(detections['detection_scores'][0].numpy()[index]))
                str_pred.append('0 0 1 1')
            else:
                if(not str_pred):
                    predicts.append('negative 1 0 0 1 1')
                else:
                    predicts.append(' '.join(str_pred))
                break
    else:
        str_pred = []
        for index in range(len(detections['detection_scores'][0])):
            if(detections['detection_scores'][0].numpy()[index] > 0.5):
                str_pred.append(category_index[detections['detection_classes'][0].numpy(
                ).astype(np.int32)[index]]['name'])
                str_pred.append(
                    str(detections['detection_scores'][0].numpy()[index]))
                str_pred.append(
                    str(int(np.around(detections['detection_boxes'][0].numpy()[index][0]*width))))
                str_pred.append(
                    str(int(np.around(detections['detection_boxes'][0].numpy()[index][1]*height))))
                str_pred.append(
                    str(int(np.around(detections['detection_boxes'][0].numpy()[index][2]*width))))
                str_pred.append(
                    str(int(np.around(detections['detection_boxes'][0].numpy()[index][3]*height))))
            else:
                if(not str_pred):
                    predicts.append('negative 1 0 0 1 1')
                else:
                    predicts.append(' '.join(str_pred))
                break

In [ ]:
submission_df['PredictionString'] = predicts
submission_df.head()